In [ ]:
"""
依赖自检
"""

import sys
import importlib.util

print(f"{'='*10} 执行依赖自检 {'='*10}")

required_packages = [
    ("torch", "PyTorch" ),
    ("torchvision", "TorchVision"),
    ("diffusers", "Diffusers"),
    ("matplotlib", "Matplotlib")
]

missing_packages = []

for package_name, display_name in required_packages:
    if importlib.util.find_spec(package_name) is None:
        missing_packages.append(package_name)
    else:
        module = __import__(package_name)
        version = getattr(module, '__version__', '未知版本')
        print(f"{display_name}: {version}")

if missing_packages:
    print(f"\n【ERROR】缺少以下依赖包: {', '.join(missing_packages)}")
    sys.exit(1)
else:
    try:
        import torch
        import torchvision
        from torch import nn
        from torch.nn import functional as f
        from torch.utils.data import DataLoader
        from diffusers import DDPMScheduler, UNet2DModel
        from matplotlib import pyplot as plt
    except Exception as e:
        print(f"【ERROR】导入发生未知错误")
        sys.exit(1)

print(f"{'='*10} 依赖自检完成 {'='*10}")

In [ ]:
"""
Device 自检
Apple Silicon：优先 mps，否则 cpu
NVIDIA：优先 cuda，否则 cpu
"""

print(f"{'='*10} 执行硬件自检 {'='*10}")

try:
    if torch.backends.mps.is_available():
        device_name = "mps"
        if torch.backends.mps.is_built():
            print("【INFO】Use Apple Silicon (MPS)")
    elif torch.cuda.is_available():
        device_name = "cuda"
        print("【INFO】Use NVIDIA")
    else:
        device_name = "cpu"
        print("【INFO】Use CPU")

    device = torch.device(device_name)
    x = torch.ones(1).to(device)  # 在CPU中创建张量，并将其移动至device，赋值给x
    print(f"{device} 可以使用")

except Exception as e:
    print(f"【ERROR】{e} 设备无法使用")
    device = torch.device("cpu")

print(f"{'='*10} 硬件自检完成 {'='*10}")

In [ ]:
"""
数据集测试
"""

# 数据集
dataset = torchvision.datasets.MNIST(
                                root="../data/datasets",
                                train=True,                                  # 使用训练集，False为测试集
                                download=True,                               # 下载数据集
                                transform=torchvision.transforms.ToTensor()  # 将图像转换为张量
                                )

# 为数据集创建数据加载器
dataset_loader = DataLoader(
                dataset,        # 要加载的数据对象
                batch_size=16,  # 每次迭代加载的样本数量
                shuffle=True    # 打乱数据顺序
                )

# 从加载器中取出第一批数据
x, y = next(iter(dataset_loader))
print('Input shape:', x.shape)
print('Labels :', y)
plt.imshow(torchvision.utils.make_grid(x)[0], cmap = 'gray')  # 以单通道取出所有图像，拼接成大图并用灰度显示

> 我们可以引入一个变量来手动控制内容“损坏”的程度。  
> 如下方的代码所示。其中 `amount` 是我们引入的、人为控制的变量，`x` 为图像张量，`noise` 为噪声，`noise_x` 为加了噪声后的张量。

`noised_x = (1-amount) * x + amount * noise`

可以看出，当 `amount` = 0 时，`noise_x` 与原始张量相同；当 `amount` = 1 时，`noise_x` 则为纯粹的噪声。

这种方法在数学上叫做线性插值。

当然，`amount` 的值不是手动填写的，那样训练就太低效了。在深度学习中，我们通常不是一张一张地处理图片，而是成批（Batch）处理，我们通常会为这一批次里的每一张图随机生成一个不同的损坏程度。

`x` (图像张量) 的形状通常是 `[Batch_size, Channels, Height, Width]`。
例如在上面的代码中，输出的 Input shape 为[16, 1, 28, 28]，表示一共有 16 张 1 通道的 28x28 图片。

问题在于如果 `amount` 是 1 维的随机数，那么它直接乘以 4 维的张量会发生错误。  
为了让 `amount` 能和 `x` 相乘，我们需要把 `amount` 从 1 维变成 4 维，形状变为 `[ , , , ]`。

于是，我们就需要用到 PyTorch 的 `.view()` 方法来改变张量的形状。

`.view(-1, 1, 1, 1)` 内有几个参数，其中 -1 表示自动计算一个批次的数量，其余的 1 都表示自动适配图片规格。其实整个方法都是起自动适配的作用。

---

In [ ]:
# 绘制输入数据
fig, axs = plt.subplots(2, 1, figsize=(12, 5))  # 画布行数，画布列数，画布大小。plt.subplots返回两个方法，第一个是画布对象fig，第二个是子图对象axs
plt.subplots_adjust(hspace=0.4)  # 扩大子图间距

axs[0].set_title('Input Images')
axs[0].imshow(torchvision.utils.make_grid(x)[0], cmap='gray')  # 以单通道取出所有图像，拼接成大图并用灰度显示

# 加入噪声
amount = torch.linspace(0, 1, x.shape[0])  # 在指定的范围内，生成一组等距离的数字，数量与x的Batch_size相同
noised_x = corrupt(x, amount)

# 绘制加入噪声后的图像
axs[1].set_title('Corrupted Images (--- amount increases -->)')
axs[1].imshow(torchvision.utils.make_grid(noised_x)[0], cmap='gray')

> 我们可以发现，当噪声量接近 1 时，数据看起来像是随机的噪声图。